In [6]:
"""complete basics"""


import tensorflow as tf

x=tf.constant(40,name="x")
y=tf.constant(50,name="y")
z = tf.Variable(x+y)

init=tf.initialize_all_variables()

sess = tf.Session()
sess.run(init);
print(sess.run(z))

90


In [1]:
"""hello world"""

import tensorflow as tf

hello=tf.constant("hello world",name="hello")


init=tf.initialize_all_variables()

sess = tf.Session()
sess.run(init);
print(sess.run(hello))

b'hello world'


In [12]:
"""complete basics- better method of running session"""


import tensorflow as tf

x=tf.constant(40,name="x")
y=tf.constant(50,name="y")
z = tf.Variable(x+y)

init=tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    #all the stuff you need to do should be here
    print(sess.run(z))

90


In [26]:

"""slight expansion - arrays"""
#it appears thet there is no difference between methods of running the session

import tensorflow as tf

x=tf.constant([15,16,17],name="x")
y=tf.constant(10,name="y")
z = tf.Variable(x+y)

init=tf.initialize_all_variables()

sess = tf.Session() 
sess.run(init);
print(sess.run(z))

[25 26 27]


In [1]:
"""basic saver example, requires kernel reset after saving, before loading"""
import tensorflow as tf

number = tf.Variable(1.0,name="number")

init_op = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init_op)
    saver = tf.train.Saver()
    save_path = saver.save(sess, "/tmp/model.ckpt")
    print("Model saved in file: %s" % save_path)



Model saved in file: /tmp/model.ckpt


In [1]:

"""basic restoring example"""
import tensorflow as tf

number = tf.Variable(5.0,name="number")

init_op = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init_op)
    print(sess.run(number))
    saver = tf.train.Saver()
    saver.restore(sess, "/tmp/model.ckpt")
    print(sess.run(number))

5.0
1.0


In [4]:
"""simple optimizer: finding minimum of y=x+2x^2"""

import tensorflow as tf
import numpy as np

a =tf.constant(1.0,name="a")
b=tf.constant(2.0,name="b")

x=tf.Variable(1.0,name="x")

out = tf.mul(a,x)+tf.mul(b,tf.mul(x,x))

train_op = tf.train.GradientDescentOptimizer(0.001).minimize(out)

init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        sess.run(train_op)
        
    print("x:",sess.run(out))
    print("y(x):",sess.run(x))

x: -0.123968
y(x): -0.227288


In [25]:
"""simple 2d optimizer: finding minimum of y=x+2x^2-z+2z^2, with use of arrays"""

import tensorflow as tf
import numpy as np

a =tf.constant(1.0,name="a")
b=tf.constant(2.0,name="b")
c=tf.constant(-1.0,name="c")
d=tf.constant(2.0,name="d")

vars=tf.Variable([1.0,1.0],name="vars") #vars contains x and z

out = tf.mul(a,vars[0])+tf.mul(b,tf.mul(vars[0],vars[0]))+tf.mul(c,vars[1])+tf.mul(d,tf.mul(vars[1],vars[1]))

train_op = tf.train.GradientDescentOptimizer(0.001).minimize(out)

init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        sess.run(train_op)
        
    print(sess.run(out)) # min y
    print(sess.run(vars[0]))# x
    print(sess.run(vars[1]))# z

-0.248597
-0.227288
0.263627


In [5]:
"""linear regression, partially from examples"""


#
import tensorflow as tf
import numpy
import matplotlib.pyplot as plt
rng = numpy.random

#
# Training Data
train_X = numpy.asarray([3.3,4.4,5.5,6.71,6.93,4.168,9.779,6.182,7.59,2.167,7.042,10.791,5.313,7.997,5.654,9.27,3.1])
train_Y = numpy.asarray([1.7,2.76,2.09,3.19,1.694,1.573,3.366,2.596,2.53,1.221,2.827,3.465,1.65,2.904,2.42,2.94,1.3])

n_samples = train_X.shape[0]

#
# numbers
epoch_num=1000;
trainig_rate=0.001;

# 
# model
X=tf.placeholder("float")
Y=tf.placeholder("float")

W=tf.Variable(rng.randn(),name="weight")
b=tf.Variable(rng.randn(),name="bias")

pred = tf.add(tf.mul(X, W), b)

#
# Mean squared error
cost = tf.reduce_sum(tf.pow(pred-Y, 2))/(2*n_samples)

train_op = tf.train.GradientDescentOptimizer(trainig_rate).minimize(cost)


init=tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    for i in range(epoch_num):
        for (x, y) in zip(train_X, train_Y):
            sess.run(train_op, feed_dict={X: x, Y: y})
        
    print(sess.run(W))
    print(sess.run(b))
    
    plt.plot(train_X, train_Y, 'ro', label='Original data')
    plt.plot(train_X, sess.run(W) * train_X + sess.run(b), label='Fitted line')
    plt.legend()
    plt.show()

0.335843
0.199692


In [21]:
"""completely basic network"""

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

#numbers:
epoch_num=15
training_rate=0.001
batch_size=100

#placeholders & variables
Data=tf.placeholder("float32",[None,784])
Labels=tf.placeholder("float32",[None,10])

Weights_1=tf.Variable(tf.random_normal([784,256],stddev=0.01))
Weights_2=tf.Variable(tf.random_normal([256,10],stddev=0.01))

Biases_1=tf.Variable(tf.random_normal([256],stddev=0.01))
Biases_2=tf.Variable(tf.random_normal([10],stddev=0.01))

#model
def network(x,Weights1,Weights2,Biases1,Biases2):
    layer_1=tf.add(tf.matmul(x,Weights1),Biases1)
    layer_1=tf.nn.relu(layer_1)
    
    layer_2=tf.add(tf.matmul(layer_1,Weights2),Biases2)
    return layer_2


pred =network(Data,Weights_1,Weights_2,Biases_1,Biases_2)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, Labels))

train_op = tf.train.GradientDescentOptimizer(training_rate).minimize(cost)

init =tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epoch_num):
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run([train_op, cost], feed_dict={Data: batch_x, Labels: batch_y})
    
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Labels, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print ("Accuracy:", accuracy.eval({Data: mnist.test.images, Labels: mnist.test.labels}))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Accuracy: 0.7954


In [23]:
"""completely basic multilayer"""

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

#numbers:
epoch_num=15
training_rate=0.05
batch_size=100

#placeholders & variables
Data=tf.placeholder("float32",[None,784])
Labels=tf.placeholder("float32",[None,10])

Weights_1=tf.Variable(tf.random_normal([784,256],stddev=0.01))
Weights_2=tf.Variable(tf.random_normal([256,256],stddev=0.01))
Weights_3=tf.Variable(tf.random_normal([256,10],stddev=0.01))

Biases_1=tf.Variable(tf.random_normal([256],stddev=0.01))
Biases_2=tf.Variable(tf.random_normal([256],stddev=0.01))
Biases_3=tf.Variable(tf.random_normal([10],stddev=0.01))

#model
def network(x,Weights1,Weights2,Weights3,Biases1,Biases2,Biases3):
    layer_1=tf.add(tf.matmul(x,Weights1),Biases1)
    layer_1=tf.nn.relu(layer_1)
 
    layer_2=tf.add(tf.matmul(layer_1,Weights2),Biases2)
    layer_2=tf.nn.relu(layer_2)
    
    layer_3=tf.add(tf.matmul(layer_2,Weights3),Biases3)
    return layer_3


pred =network(Data,Weights_1,Weights_2,Weights_3,Biases_1,Biases_2,Biases_3)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, Labels))

train_op = tf.train.GradientDescentOptimizer(training_rate).minimize(cost)

init =tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epoch_num):
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run([train_op, cost], feed_dict={Data: batch_x, Labels: batch_y})
    
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Labels, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print ("Accuracy:", accuracy.eval({Data: mnist.test.images, Labels: mnist.test.labels}))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Accuracy: 0.9702


In [9]:
"""completely basic multilayer -with non relu activation function"""

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

#numbers:
epoch_num=15
training_rate=0.05
batch_size=100

#placeholders & variables
Data=tf.placeholder("float32",[None,784])
Labels=tf.placeholder("float32",[None,10])

Weights_1=tf.Variable(tf.random_normal([784,256],stddev=0.01))
Weights_2=tf.Variable(tf.random_normal([256,256],stddev=0.01))
Weights_3=tf.Variable(tf.random_normal([256,10],stddev=0.01))

Biases_1=tf.Variable(tf.random_normal([256],stddev=0.01))
Biases_2=tf.Variable(tf.random_normal([256],stddev=0.01))
Biases_3=tf.Variable(tf.random_normal([10],stddev=0.01))

#model
def network(x,Weights1,Weights2,Weights3,Biases1,Biases2,Biases3):
    layer_1=tf.add(tf.matmul(x,Weights1),Biases1)
    layer_1=tf.nn.relu6(layer_1)#possible instead: sigmoid (80% acc), relu (90-96 % accuracy) 
 
    layer_2=tf.add(tf.matmul(layer_1,Weights2),Biases2)
    layer_2=tf.nn.relu6(layer_2)
    
    layer_3=tf.add(tf.matmul(layer_2,Weights3),Biases3)
    return layer_3


pred =network(Data,Weights_1,Weights_2,Weights_3,Biases_1,Biases_2,Biases_3)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, Labels))

train_op = tf.train.GradientDescentOptimizer(training_rate).minimize(cost)

init =tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epoch_num):
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run([train_op, cost], feed_dict={Data: batch_x, Labels: batch_y})
        
    
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Labels, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print ("Accuracy:", accuracy.eval({Data: mnist.test.images, Labels: mnist.test.labels}))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Accuracy: 0.9654


In [16]:
"""completely basic multilayer -with dropout & Adam Optimizer"""

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

#numbers:
epoch_num=10
training_rate=0.001
batch_size=100

#placeholders & variables
Data=tf.placeholder("float32",[None,784])
Labels=tf.placeholder("float32",[None,10])

Weights_1=tf.Variable(tf.random_normal([784,256],stddev=0.01))
Weights_2=tf.Variable(tf.random_normal([256,256],stddev=0.01))
Weights_3=tf.Variable(tf.random_normal([256,10],stddev=0.01))

Biases_1=tf.Variable(tf.random_normal([256],stddev=0.01))
Biases_2=tf.Variable(tf.random_normal([256],stddev=0.01))
Biases_3=tf.Variable(tf.random_normal([10],stddev=0.01))



#dropout
keep_prob = tf.placeholder("float")


#model
def network(x,Weights1,Weights2,Weights3,Biases1,Biases2,Biases3):
    layer_1=tf.add(tf.matmul(x,Weights1),Biases1)
    layer_1=tf.nn.relu6(layer_1)#possible instead: sigmoid (80% acc), relu (90-96 % accuracy) 
 
    layer_2=tf.add(tf.matmul(layer_1,Weights2),Biases2)
    layer_2=tf.nn.relu6(layer_2)
    
    drop = tf.nn.dropout(layer_2, keep_prob)
    
    layer_3=tf.add(tf.matmul(drop,Weights3),Biases3)
    return layer_3


pred =network(Data,Weights_1,Weights_2,Weights_3,Biases_1,Biases_2,Biases_3)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, Labels))

train_op = tf.train.AdamOptimizer(training_rate).minimize(cost)

init =tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epoch_num):
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run([train_op, cost], feed_dict={Data: batch_x, Labels: batch_y,keep_prob: 1.0 })
        
    
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Labels, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

    print ("Accuracy-test:", accuracy.eval({Data: mnist.test.images, Labels: mnist.test.labels,keep_prob: 1.0}))
    print ("Accuracy-train:", accuracy.eval({Data: mnist.train.images, Labels: mnist.train.labels,keep_prob: 1.0}))
    print ("Accuracy-eval:", accuracy.eval({Data: mnist.validation.images, Labels: mnist.validation.labels,keep_prob: 1.0}))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Accuracy-test: 0.977
Accuracy-train: 0.995073
Accuracy-eval: 0.9764


In [13]:
"""completely basic multilayer- overfiting research -Adam Optimizer, no dropout, eval on mnist.test/mnist.train/mnist.eval"""

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

#numbers:
epoch_num=10
training_rate=0.001
batch_size=100

#placeholders & variables
Data=tf.placeholder("float32",[None,784])
Labels=tf.placeholder("float32",[None,10])

Weights_1=tf.Variable(tf.random_normal([784,256],stddev=0.01))
Weights_2=tf.Variable(tf.random_normal([256,256],stddev=0.01))
Weights_3=tf.Variable(tf.random_normal([256,10],stddev=0.01))

Biases_1=tf.Variable(tf.random_normal([256],stddev=0.01))
Biases_2=tf.Variable(tf.random_normal([256],stddev=0.01))
Biases_3=tf.Variable(tf.random_normal([10],stddev=0.01))





#model
def network(x,Weights1,Weights2,Weights3,Biases1,Biases2,Biases3):
    layer_1=tf.add(tf.matmul(x,Weights1),Biases1)
    layer_1=tf.nn.relu6(layer_1)#possible instead: sigmoid (80% acc), relu (90-96 % accuracy) 
 
    layer_2=tf.add(tf.matmul(layer_1,Weights2),Biases2)
    layer_2=tf.nn.relu6(layer_2)
    
    
    
    layer_3=tf.add(tf.matmul(layer_2,Weights3),Biases3)
    return layer_3


pred =network(Data,Weights_1,Weights_2,Weights_3,Biases_1,Biases_2,Biases_3)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, Labels))

train_op = tf.train.AdamOptimizer(training_rate).minimize(cost)

init =tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epoch_num):
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run([train_op, cost], feed_dict={Data: batch_x, Labels: batch_y })
        
    
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Labels, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print ("Accuracy-test:", accuracy.eval({Data: mnist.test.images, Labels: mnist.test.labels}))
    print ("Accuracy-train:", accuracy.eval({Data: mnist.train.images, Labels: mnist.train.labels}))
    print ("Accuracy-eval:", accuracy.eval({Data: mnist.validation.images, Labels: mnist.validation.labels}))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Accuracy-test: 0.9783
Accuracy-train: 0.995982
Accuracy-eval: 0.9812


In [ ]:
"""tensorflow official example- for comparison (mnist for beginners)"""


global graph_xes
global graph_ys

graph_xes=[]
graph_ys=[]

def print_results():
    global graph_xes
    global graph_ys
    %matplotlib inline
    plt.plot(graph_xes,graph_ys)
    plt.show()
    graph_xes=[]
    graph_ys=[]


def doStuff(in_range, batch_size):
    global graph_xes
    global graph_ys
    #different layer setup
    x = tf.placeholder(tf.float32, [None, 784])

    W = tf.Variable(tf.zeros([784, 10]))
    b = tf.Variable(tf.zeros([10]))

    y = tf.nn.softmax(tf.matmul(x, W) + b)

    y_ = tf.placeholder(tf.float32, [None, 10])
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
    #different annotation of cost function

    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

    init = tf.initialize_all_variables()
    sess = tf.Session()
    sess.run(init)

    for i in range(in_range):
      batch_xs, batch_ys = mnist.train.next_batch(batch_size)
      sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    print("in range ",in_range,"batch size :",batch_size)
    result = sess.run(accuracy, feed_dict={x: mnist.validation.images, y_: mnist.validation.labels})
    graph_xes.append(batch_size)
    graph_ys.append(result)
    print(result)
    
    
    tf.reset_default_graph()
    

  
for x in range(1,30):
    doStuff(1000,x*10)

print_results()    



In [1]:
"""saver example -network to be saved"""
#requires kernel resart to be later loaded
#utilizes below network:
"""completely basic multilayer -with Adam Optimizer"""

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

#numbers:
epoch_num=30
training_rate=0.001
batch_size=100

#placeholders & variables
Data=tf.placeholder("float32",[None,784])
Labels=tf.placeholder("float32",[None,10])

Weights_1=tf.Variable(tf.random_normal([784,256],stddev=0.01))
Weights_2=tf.Variable(tf.random_normal([256,256],stddev=0.01))
Weights_3=tf.Variable(tf.random_normal([256,10],stddev=0.01))

Biases_1=tf.Variable(tf.random_normal([256],stddev=0.01))
Biases_2=tf.Variable(tf.random_normal([256],stddev=0.01))
Biases_3=tf.Variable(tf.random_normal([10],stddev=0.01))

#model
def network(x,Weights1,Weights2,Weights3,Biases1,Biases2,Biases3):
    layer_1=tf.add(tf.matmul(x,Weights1),Biases1)
    layer_1=tf.nn.relu(layer_1)
 
    layer_2=tf.add(tf.matmul(layer_1,Weights2),Biases2)
    layer_2=tf.nn.relu(layer_2)
    
    layer_3=tf.add(tf.matmul(layer_2,Weights3),Biases3)
    return layer_3


pred =network(Data,Weights_1,Weights_2,Weights_3,Biases_1,Biases_2,Biases_3)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, Labels))

train_op = tf.train.AdamOptimizer(training_rate).minimize(cost)

init =tf.initialize_all_variables()

#complete network saver
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epoch_num):
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run([train_op, cost], feed_dict={Data: batch_x, Labels: batch_y})
    
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Labels, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print ("Accuracy:", accuracy.eval({Data: mnist.test.images, Labels: mnist.test.labels}))
    save_path = saver.save(sess, "/tmp/model.ckpt")
    print("Model saved in file: %s" % save_path)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Accuracy: 0.9764
Model saved in file: /tmp/model.ckpt


In [1]:
"""loader example -network to be loaded"""
#requires kernel resart before loading, uses above example to save
#utilizes below network:
"""completely basic multilayer -with Adam Optimizer"""

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

#numbers:
epoch_num=30
training_rate=0.001
batch_size=100

#placeholders & variables
Data=tf.placeholder("float32",[None,784])
Labels=tf.placeholder("float32",[None,10])

Weights_1=tf.Variable(tf.random_normal([784,256],stddev=0.01))
Weights_2=tf.Variable(tf.random_normal([256,256],stddev=0.01))
Weights_3=tf.Variable(tf.random_normal([256,10],stddev=0.01))

Biases_1=tf.Variable(tf.random_normal([256],stddev=0.01))
Biases_2=tf.Variable(tf.random_normal([256],stddev=0.01))
Biases_3=tf.Variable(tf.random_normal([10],stddev=0.01))

#model
def network(x,Weights1,Weights2,Weights3,Biases1,Biases2,Biases3):
    layer_1=tf.add(tf.matmul(x,Weights1),Biases1)
    layer_1=tf.nn.relu(layer_1)
 
    layer_2=tf.add(tf.matmul(layer_1,Weights2),Biases2)
    layer_2=tf.nn.relu(layer_2)
    
    layer_3=tf.add(tf.matmul(layer_2,Weights3),Biases3)
    return layer_3


pred =network(Data,Weights_1,Weights_2,Weights_3,Biases_1,Biases_2,Biases_3)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, Labels))



init =tf.initialize_all_variables()

#complete network saver
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    saver.restore(sess, "/tmp/model.ckpt")
    print("Model restored.")
    #model doesnt have any training code, yet it works after loading parameters
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Labels, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print ("Accuracy:", accuracy.eval({Data: mnist.test.images, Labels: mnist.test.labels}))


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Model restored.
Accuracy: 0.9764
